In [1]:
import os
#turning on local GPUs
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # o refers to one device, 1 refers to 2 gpus

#importing dependencies


In [2]:
#env
#C:\Users\cgree\.conda\envs\keras-gpu\Lib\site-packages\gym\envs\toy_text
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import os # for creating directories
from keras.callbacks import TensorBoard 
import keras.backend.tensorflow_backend as backend
import tensorflow as tf
#import time
from tqdm import tqdm
import datetime
from keras.models import load_model
from keras.models import model_from_json
import json
import os
os.chdir(r'C:\Users\cgree\Documents\Astra\Space_weather5_22\weakley_all')

env = gym.make('GuessingGame-v0') # initialise environment


Using TensorFlow backend.
C:\Users\cgree\Documents\Astra\Space_weather5_22\number_tracker.py:49: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  np_graph_data = self.graph_data.as_matrix(columns=self.graph_data.columns[1:])


In [27]:
#load model
LOAD_MODEL = 'X://Final_RL//models//24X4____11.00max____5.10avg___-0.00min__311943.model' #None #'24X4____11.00max____5.40avg____0.99min__607291.model' #'X:\Final_RL\models\model_weights.h5'  #'X://Final_RL//models//24X4____-0.00max___-1.00avg___-2.00min__284576.h5' 
#None #'X://Final_RL//models//24X4____11.00max____3.39avg____0.99min__870654.model'

#LOAD_MODEL = r'C:\Users\cgree\Documents\Astra\Space_weather5_22\weakley_all\model_output\guessinggame\weights_2700.hdf5'


#modelx = agent.load(

In [28]:
#we need to change the four from hard code later
#this is just the way we plan to move.
'''Data to include  would say: AL, AU, SymH, Bz, and Kp might be a good initial, include local time-magnetic local time, lat/long, in correlation to the earth and sun'''


state_size = 4 #env.observation_space.n #we changed this to one because ther is only one input for state as of now
# in order to get the four the states need to be gic, and three other variables proabaly
print(state_size)

action_size = env.action_space.shape[0]
print(action_size)

batch_size = 32 #32 #running thing concurently really matters if running gpu, makes running the system more efficent by updating weights for more at one time

n_episodes = 4001 #1001 # n games we want agent to play (default 1001) the more games we play the more data we get for training
#each episode we learn little bit and we pass it on to train with.

output_dir = r'X:\Final_RL\model_output'

#tensorboard --logdir=C:\Users\cgree\Documents\Astra\Space_weather5_22\weakley_all\logs
#covnet dementions
MODEL_NAME = "24X4"
MODEL_NAME_NN = "basic_NN"
ep_rewards = [-2]
MIN_REWARD = -2 #for model save
MIN_REPLAY_MEMORY_SIZE = 2_000
DISCOUNT = 0.99 # DISCOUNT FUTURE REWARDS
UPDATE_TARGET_EVERY = 5
MINIBATCH_SIZE = 64
epsilon = 1
#stats settings
AGGREGATE_STATS_EVERY = 50 #EPISODES
Z = datetime.datetime.now()
Z = Z.strftime("%f")

4
1


In [29]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [30]:
def does_the_csv_exist():
    # if file does not exist write header 
    if not os.path.isfile(r'C:\Users\cgree\Documents\Astra\Space_weather5_22\weakley_all\guess.csv'):
        d = {'guess': []}
        df = pd.DataFrame(data=d)
        df.to_csv('guess.csv',index=False)
    else: # else it exists so append without writing the header
        pass
does_the_csv_exist()


In [31]:
# Create models folder
if not os.path.isdir('models'):
    os.makedirs('models')

In [32]:
# For more repetitive results
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)
# Own Tensorboard class

In [33]:
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.create_file_writer(self.log_dir)
        #self.log_dir = f"logs/{MODEL_NAME}-{int(time.time())}"
        self._log_write_dir = self.log_dir

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)

    #physically writes to the log files
    def _write_logs(self, logs, index):
        with self.writer.as_default():
            for name, value in logs.items():
                #tf.summary.scalar('loss',stats['loss'], step=self.step)
                tf.summary.scalar(name, value, step=index)
                self.step += 1
                self.writer.flush()
    
    def _write_logs(self, logs, index):
        with self.writer.as_default():
            for name, value in logs.items():
                #tf.summary.scalar('loss',stats['loss'], step=self.step)
                tf.summary.scalar(name, value, step=index)
                self.step += 1
                self.writer.flush()



In [34]:
'''Define the Agent'''

class DQNAgent:
    def __init__(self): #, state_size, action_size):
        self.state_size = 4 #state_size
        self.action_size = 1 #action_size
        self.memory = deque(maxlen=200500) # double-ended queue; acts like list, but elements can be added/removed from either end 
        #memory saved from episodes. saves us from going over the full episode and looks at samples randomly thought game rather than contious
        #sampling will also add diversity so it doesnt always start with going up or down. deque is a list that cuts down last 2000 memories
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 0 #be sure to change to 0 for trained models #1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 #0.99975 #0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.001 #0.0385 #0.01 # minimum amount of random exploration permitted
        self.learning_rate = 1e-3 # rate at which NN adjusts models parameters via SGD to reduce cost 
        #this method gets trained every step
        self.model = self._build_model() # private method below (basically a regular net)

        #we are going to have two models due to this.
        #This model goes all over the place because it fits for every single step. 
        #this model we will do a .predict every model instead of doing a .fit every step like the self.model
        #this model predicts everytime to give the model some type of consistency to leverage off of.
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())

        self.tensorboard = ModifiedTensorBoard(log_dir="X:\Final_RL\logs\{}-{}".format(MODEL_NAME, int(datetime.datetime.now().strftime("%f"))))#log_dir="logs\{}-{}".format(MODEL_NAME, int(Z)))#(log_dir=f"logs/{MODEL_NAME}-{int(time.time())}")
        self.target_update_counter = 0

    def _build_model(self):
        
        if LOAD_MODEL is not None:
            with open('model_in_json.json','r') as f:
                model_json = json.load(f)
                model = model_from_json(model_json)
                model.load_weights(LOAD_MODEL)
#             print(f"Loading {LOAD_MODEL}")
#             model = load_model(LOAD_MODEL)
#             print(f'Model {LOAD_MODEL} loaded:')
        else:
        #LSTM model https://papers.nips.cc/paper/1953-reinforcement-learning-with-long-short-term-memory.pdf
        # neural net to approximate Q-value function:
            model = Sequential()
            model.add(Dense(24, input_dim=self.state_size, activation='relu')) # input layer; states as input
            model.add(Dropout(0.2)) #dropout
            model.add(Dense(12, activation='relu')) # hidden layer OG 24
            model.add(Dropout(0.2))
            model.add(Dense(self.action_size, activation='linear')) # 400 actions, so 400 output neurons: -4 and 4 (L/R)
            model.compile(loss='mse',
                          optimizer=Adam(lr=self.learning_rate),
                          metrics=['accuracy']) #original loss='mse'
            model.summary()
            #log_dir= f'models/{MODEL_NAME_NN}__{int(time.time())}.models'

            #log_dir=f"logs/{MODEL_NAME_NN}-{int(time.time())}"
            #log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        return model

    #replay memory add steps data to memory replay araray with daques
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        state = np.expand_dims(state, axis = 0)
        act_values = self.model.predict(np.array(state)) # if not acting randomly, predict reward value based on current state
        #AttributeError'int' object has no attribute 'ndim'
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        #start training only if certain number of sample is already saved
        if len(self.memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        #get current states from minibatch, then query NN model for Q values
        state = np.array([transition[0] for transition in minibatch])
        current_qs_list = self.model.predict(state)

        new_current_states = np.array([transition[3] for transition in minibatch])
        future_qs_list = self.target_model.predict(new_current_states)
        x = []
        y = []

        for index, (state, action, reward, new_current_states, done) in enumerate(minibatch):

            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            x.append(state)
            y.append(current_qs)
            print(current_qs)

        self.model.fit(np.array(x), np.array(y), batch_size = MINIBATCH_SIZE, 
        verbose=0, shuffle=False, callbacks=[self.tensorboard])

        #updating to derermin if we want to update target_model yet
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights)
            self.target_update_counter = 0

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        '''
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                next_state = np.expand_dims(next_state, axis = 0) #expected dense_1_input to have 2 dimensions, # Remove this line below, as it would set back shape to 3
                #print(self.model.predict(np.array(next_state)))
                target = (reward + self.gamma * np.amax(self.model.predict(np.array(next_state))[0])) # (maximum target Q based on future action a')
            state = np.expand_dims(state, axis = 0) # repeat process to add dim
            #print(self.model.predict(np.array(state))) #slap on np.array to convert from int
            #update Q value for given state
            target_f = self.model.predict(state) #current qs list # approximately map current state to future discounted reward
            target_f[0][action] = target
            #log_dir= 'models/' + MODEL_NAME_NN + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            x.append(state)
            y.append(target_f)
            #str(log_dir)
            #log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            print(state)
            print(target_f)
            print(x)
            print(y)
            print(np.array)
            #tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1) #tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            self.model.fit(x, y, batch_size = 64, epochs=1, verbose=0, shuffle=False, callbacks=[self.tensorboard])
            #self.model.fit(state, target_f, epochs=1, verbose=0, callbacks=[self.tensorboard]) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        '''

    #def load(self, name):
    #    self.model.load_weights(name)

    #def save(self, name):
    #    self.model.save_weights(name)


In [35]:
'''initalize agent'''
agent = DQNAgent() #state_size, action_size) # initialise agent
'''Interact with environment'''
done = False #default is game has not ended.
for e in tqdm(range(1, n_episodes + 1 ), ascii=True, unit='eposodes'): # iterate over new episodes of the game

    # Update tensorboard step every episode
    agent.tensorboard.step = e
    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    step = 1

    # reset state at start of each new episode of the game
    state = env.reset() 
    '''
    #reset number and start iteration until episode ends
    done = False
    while not done:
        
        #query a model for Q values
        if np.random.random() > epsilon:
            #get action from Q table
            action = agent.act(np.array(state))
        else:
            #get random action
            action = np.random.randint(0, env.action_space.shape[0])
        
        next_state, reward, done, info = env.step(np.array(action))
        
        #transform new continous state to new discrete state and count reward
        episode_reward += reward
        
        #every step we update replay memory and train main network
        agent.remember(state, action, reward, next_state, done)
        agent.replay(batch_size)
        
        state = next_state
        step += 1
    '''
    for time in range(50):  # time represents a frame of the game; goal is to stay within the 1% range. once its done it can go on for only 5000 steps e.g., 500 or 5000 timesteps
#         env.render() #moving my animation to render
        action = agent.act(np.array(state)) # action is 0 - 4 (move up, down, equal or no guess); decide on one or other here

        next_state, reward, done, info = env.step(np.array(action)) #env.step(np.array([guess])) # agent interacts with env, gets feedback; 4 state data points, e.g., high or low - observation - later the variables and  done, reward        
     
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state # set "current state" for upcoming iteration to the current next state        
        if done: # episode ends if agent is outside 1% or we reach timestep 5000
            break # exit loop
    try:
        if len(agent.memory) > batch_size:
            agent.replay(batch_size) # train the agent by replaying the experiences of the episode
    except RuntimeError:
        print('RuntimeError is raised')
        pass
        
    #if e % 100 == 0:
    #    agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5") #saving every 50 episode. we can decide which model weight to hold onto
    # print("next state all the data")
    # print(state)
   
   
   #append episode reward to a list and log stats (every given number of episodes)
    ep_rewards.append(reward)
    if not e % AGGREGATE_STATS_EVERY or e == 1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
        max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
        agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=agent.epsilon)

        #save model, but only when min reward is greater or equal a set value
        # try:
        
        if average_reward >= MIN_REWARD: #>= MIN_REWARD:
            import json
            # lets assume `model` is main model 
            model_json = agent.model.to_json()
            with open("model_in_json.json", "w") as json_file:
                json.dump(model_json, json_file)

            agent.model.save_weights(f'X:\Final_RL\models2\{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(datetime.datetime.now().strftime("%f"))}.h5')
           
            
            
            #agent.model.save_weights(f'X:\Final_RL\models\{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(datetime.datetime.now().strftime("%f"))}.model')
            #agent.save(f'X:\Final_RL\models\{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(datetime.datetime.now().strftime("%f"))}.h5')
                #agent.model.save((f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model'))
            #agent.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')
        # except AttributeError as e:
        #     print(e)
        #     pass


        #tensorboard --port=6006 --logdir=X:\Final_RL\models
        #tensorboard --port=6007 --logdir=X:\Final_RL\logs







  0%|          | 0/4001 [00:00<?, ?eposodes/s]



  0%|          | 1/4001 [00:01<1:23:58,  1.26s/eposodes]



  0%|          | 2/4001 [00:02<1:21:22,  1.22s/eposodes]



  0%|          | 3/4001 [00:03<1:18:18,  1.18s/eposodes]



  0%|          | 4/4001 [00:04<1:16:05,  1.14s/eposodes]



  0%|          | 5/4001 [00:05<1:14:19,  1.12s/eposodes]



  0%|          | 6/4001 [00:06<1:13:08,  1.10s/eposodes]



  0%|          | 7/4001 [00:07<1:12:18,  1.09s/eposodes]



  0%|          | 8/4001 [00:08<1:11:38,  1.08s/eposodes]



  0%|          | 9/4001 [00:09<1:10:44,  1.06s/eposodes]



  0%|          | 10/4001 [00:10<1:10:52,  1.07s/eposodes]



  0%|          | 11/4001 [00:11<1:10:00,  1.05s/eposodes]



  0%|          | 12/4001 [00:12<1:10:12,  1.06s/eposodes]



  0%|          | 13/4001 [00:14<1:10:48,  1.07s/eposodes]



  0%|          | 14/4001 [00:15<1:11:09,  1.07s/eposodes]



  0%|          | 15/4001 [00:16<1:10:31,  1.06s/eposodes]



  0%|          | 16/4001 [00:17<1:09:47,

you got it






  1%|1         | 46/4001 [00:48<1:11:24,  1.08s/eposodes]



  1%|1         | 47/4001 [00:49<1:11:00,  1.08s/eposodes]



  1%|1         | 48/4001 [00:50<1:09:40,  1.06s/eposodes]



  1%|1         | 49/4001 [00:51<1:09:08,  1.05s/eposodes]



  1%|1         | 50/4001 [00:52<1:10:04,  1.06s/eposodes]



  1%|1         | 51/4001 [00:53<1:10:23,  1.07s/eposodes]



  1%|1         | 52/4001 [00:54<1:10:12,  1.07s/eposodes]



  1%|1         | 53/4001 [00:56<1:10:50,  1.08s/eposodes]



  1%|1         | 54/4001 [00:57<1:10:59,  1.08s/eposodes]



  1%|1         | 55/4001 [00:58<1:09:41,  1.06s/eposodes]



  1%|1         | 56/4001 [00:59<1:09:25,  1.06s/eposodes]



  1%|1         | 57/4001 [01:00<1:08:47,  1.05s/eposodes]



  1%|1         | 58/4001 [01:01<1:08:07,  1.04s/eposodes]



  1%|1         | 59/4001 [01:02<1:08:17,  1.04s/eposodes]



  1%|1         | 60/4001 [01:03<1:09:31,  1.06s/eposodes]



  2%|1         | 61/4001 [01:04<1:09:29,  1.06s/eposodes]



  2%|1         | 62/

you got it






  3%|3         | 123/4001 [02:09<1:08:14,  1.06s/eposodes]

...

[0.99501866]
[0.9658978]
[0.990025]
[0.9851671]
[0.9802961]
[0.99501866]
[0.97546107]
[0.97066176]
[0.9851671]
[0.97546107]
[0.9380368]
[0.9518144]
[0.9802961]
[0.99501866]
[0.9851671]
[0.9850562]
[0.96116877]
[0.99500626]
[0.9425959]
[0.9335107]
[11.]
[11.]
[11.]
[0.9802961]
[0.9851671]
[0.9851671]
[0.97546107]
[0.9658978]
[0.9900745]
[0.9900745]
[0.96116877]
[0.97546107]
RuntimeError is raised






100%|#########9| 4000/4001 [1:11:03<00:00,  1.01eposodes/s]

[0.9335107]
[11.]
[0.9802961]
[11.]
[0.9851671]
[0.99501866]
[0.99500626]
[0.99500626]
[0.9802961]
[0.9900745]
[0.9900745]
[0.97515625]
[0.9801]
[0.99501866]
[0.9900745]
[0.9564744]
[0.9802961]
[0.9380368]
[0.99501866]
[11.]
[0.9851671]
[0.9900745]
[0.99500626]
[0.9851671]
[0.99501866]
[0.97546107]
[11.]
[0.97546107]
[0.9851671]
[0.99501866]
[0.9850562]
[0.97546107]
RuntimeError is raised






100%|##########| 4001/4001 [1:11:04<00:00,  1.07s/eposodes]

[0.9851671]
[0.9850562]
[0.97066176]
[0.9851671]
[0.970225]
[0.99500626]
[0.9658978]
[0.9802961]
[0.99501866]
[0.9802961]
[0.9564744]
[0.99501866]
[0.97546107]
[0.9802961]
[0.97066176]
[0.9850562]
[0.99501866]
[0.9518144]
[0.9658978]
[0.9802961]
[11.]
[0.9851671]
[0.9658978]
[0.97546107]
[0.99501866]
[0.9900745]
[11.]
[0.9380368]
[0.9802961]
[0.97066176]
[0.9851671]
[0.99500626]
RuntimeError is raised


In [64]:
agent.test(env, nb_episodes=5, visualize=True)

AttributeError: 'DQNAgent' object has no attribute 'test'

In [1]:
from number_tracker import number_tracking
obj = number_tracking()

In [2]:
import pandas as pd

In [3]:
dfcc= obj.new_df()

In [4]:
dfcc

0.08

In [9]:
dxxx = obj.list_for_active_chart()

In [10]:
dxxx

In [53]:
print(state)

[ 1.         -2.09675333         nan -6.04600001]


In [23]:
agent.act(state)

0

In [29]:
agent = DQNAgent(state_size, action_size)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 24)                120       
_________________________________________________________________
dropout_5 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 12)                300       
_________________________________________________________________
dropout_6 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 13        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Outpu

In [30]:
new_model = agent.load(r'C:/Users/cgree/Documents/Astra/Space_weather5_22/weakley_all/model_output/guessinggame/weights_0900.hdf5')

ValueError: Shapes (24, 12) and (24, 24) are incompatible

In [31]:
modelx = agent.load('X://Final_RL//models//24X4____11.00max____5.10avg___-0.00min__311943.model')

In [32]:
modelx.summary()

AttributeError: 'NoneType' object has no attribute 'summary'

In [ ]:
pred_test_y = model.predict([X_test], batch_size=1024, verbose=1)

In [11]:
print(state)

[ 1.         -1.18855997         nan -6.        ]


In [ ]:
self.target_model.predict(new_current_states)